In [7]:
import pandas as pd

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [8]:
import gensim
from gensim.corpora import Dictionary
from gensim.models.ldamulticore import LdaMulticore

In [9]:

import multiprocessing
import pickle

In [ ]:
#Trainning code

dataset = 'arxiv-metadata-oai-snapshot.json'

chunk_size = 1000
chunks =[]
# chia nhỏ dữ liệu rồi đọc để tránh Memory Error
for chunk in pd.read_json(dataset,lines = True,chunksize = chunk_size):
    chunks.append(chunk)

# nối list các dataframe lại thành 1 dataframe duy nhất
df = pd.concat(chunks, ignore_index=True)  



In [56]:
# lược bỏ các cột có thông tin không cần thiết để giảm lượng dữ liệu cần tương tác
df.drop(columns=['submitter','authors','comments','journal-ref','report-no','license','versions'],inplace = True)
# loại bỏ các điểm dữ liệu có không có thông tin
df.dropna(inplace=True)

In [ ]:
# sắp xếp lại dữ liệu theo thứ tự thời gian giảm dần (để khi blind_search_engine.search() thì kết quả đầu tiên sẽ là các bài báo mới nhất)
from datetime import datetime
df['update_date'] = pd.to_datetime(df['update_date'])
df_sorted = df.sort_values(by='update_date', ascending=False)

#lưu dữ liệu thành file csv
df_sorted.to_csv('new_sorted_df.csv', index=False)

In [19]:
# load df from saved_file 
chunks =[]

for chunk in pd.read_csv('sorted_df.csv', chunksize=10000):
    chunks.append(chunk)

load_df = pd.concat(chunks, ignore_index=True)

In [20]:
df = load_df

In [6]:
#tách phần abstract ra riêng để dễ dàng thao tác, huấn luyện
abstracts = load_df['abstract']


In [9]:


#downloading các tệp cần thiết cho việc tokenize, loại bỏ stopwords
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

# hàm lemmatize, tokenize và loại bỏ stopwords trong dữ liệu 
def preprocess(text):
    text = text.lower()
    words = word_tokenize(text)
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return words


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\japan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\japan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\japan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
#lemmatize, tokenize và loại bỏ stopwords đối với abstracts 
abstracts = [preprocess(text) for text in abstracts]

In [13]:
#hàm tự viết để lưu file 

def write_file(file_name: str,content):
    f = open(file_name,'w')
    for chunk in content:
        f.write(str(chunk) + '\n')
    f.close()


def read_file(file_name:str):
    import ast
    contents = []
    with open(file_name, 'r') as file:
        for line in file:
            chunk = ast.literal_eval(line)
            contents.append(chunk)
    return contents


In [9]:
#lưu file abstract để tránh phải chạy lại khi xảy ra lỗi
write_file('new_abstracts.txt',abstracts)

In [7]:
# đọc abstracts từ file đã lưu
abs = read_file('new_abstracts.txt')

In [8]:

#tạo dictionary từ dữ liệu đã qua xử lý 
dictionary = Dictionary(abs)

#loại bỏ các xuất hiện trong ít hơn 5 văn bản và các từ xuất hiện trong 50% các văn bản
dictionary.filter_extremes(no_below=5, no_above=0.5)


In [10]:
#lưu dictionary 
dictionary.save('dictionary.dict')
print("New Dictionary saved to 'dictionary.dict'")

New Dictionary saved to 'dictionary.dict'


In [21]:
#load dictionary
dictionary = Dictionary.load('dictionary.dict')
print("Dictionary loaded from 'dictionary.dict'")



Dictionary loaded from 'dictionary.dict'


In [9]:

#tạo bag of words  với mỗi một abstract ta có
bow_corpus = [dictionary.doc2bow(text) for text in abs]


In [11]:
#Save  BoW

import pickle
with open('bow_corpus.pkl', 'wb') as f:
    pickle.dump(bow_corpus, f)

print("New BoW corpus saved to 'bow_corpus.pkl'")

New BoW corpus saved to 'bow_corpus.pkl'


In [3]:
#Load Bow

import pickle

with open('bow_corpus.pkl', 'rb') as f:
    bow_corpus = pickle.load(f)

print("BoW corpus loaded from 'bow_corpus.pkl'")

BoW corpus loaded from 'bow_corpus.pkl'


In [ ]:
# chạy với 11 core và hoàn thành sau 2 tiếng 10p

import multiprocessing
from gensim.models.ldamulticore import LdaMulticore

# đặt số lượng chủ đề muốn LDA tìm ra là 80
# tại sao lại là 80? Tại vì nguồn của dữ liệu là trang Axvir có 8 chủ đề bài báo khoa học, nên đặt là 80 :D
# đặt cao quá thì ko chạy được (đã thử 400 và 200)
num_topics = 80

# passes là số lần toàn bộ tập dữ liệu sẽ được đi qua trong quá trình huấn luyện mô hình, ta đặt nó là 20
# tại sao? vì mặc định trong mô hình là 20 :(. Viết cho vui thôi chứ để cao quá máy chạy đến đời chắt
passes = 20

# thấy mọi người khuyên khi sử dụng song song thì nên có dòng "if __name__ == "__main__"" để tránh bị vòng lặp vô hạn, file tự call đến chính nó vô hạn lần
if __name__ == "__main__":
    
    num_workers = multiprocessing.cpu_count() - 1  # Use all CPUs except one

    # sử dụng LDA multicore để chạy song song dữ liệu để chạy nhanh hơn
    lda_model = LdaMulticore(
        corpus=bow_corpus,
        id2word=dictionary,
        num_topics=num_topics,
        passes=passes,
        workers=num_workers
    )

#show topics
topics = lda_model.show_topics(num_topics=num_topics, num_words=10, log=False, formatted=False)


for topic_id, topic in topics:
    print("Topic: {}".format(topic_id))
    print("Words: {}".format([word for word, _ in topic]))

In [ ]:
#save thông tin mô hình sau khi được chạy xong để không phải chạy lại lần nữa nếu vẫn sữ dụng thông tin cũ
lda_model.save('new_lda_model.model')
print(f"LDA model saved to lda_model.model")

LDA model saved to lda_model_80.model


In [5]:
from gensim.models import LdaModel
num_topics = 80

#load LDA_model
model_file = 'new_lda_model.model'
loaded_lda_model = LdaModel.load(model_file)
print("LDA model loaded")

#show topics từ model mới được load lên
topics = loaded_lda_model.print_topics(num_topics=num_topics, num_words=5)
for topic_id, topic in topics:
    print(f"Topic {topic_id}: {topic}")

lda_model = loaded_lda_model

LDA model loaded
Topic 0: 0.030*"model" + 0.027*"pattern" + 0.022*"cell" + 0.015*"population" + 0.009*"protein"
Topic 1: 0.195*"phase" + 0.111*"transition" + 0.044*"critical" + 0.025*"temperature" + 0.024*"model"
Topic 2: 0.040*"structure" + 0.028*"molecule" + 0.023*"molecular" + 0.021*"calculation" + 0.021*"electronic"
Topic 3: 0.037*"solar" + 0.021*"flux" + 0.014*"flare" + 0.012*"region" + 0.010*"cycle"
Topic 4: 0.251*"spectrum" + 0.104*"power" + 0.076*"spectral" + 0.061*"perturbation" + 0.056*"fluctuation"
Topic 5: 0.114*"trajectory" + 0.083*"glass" + 0.051*"overlap" + 0.043*"slope" + 0.024*"replica"
Topic 6: 0.045*"star" + 0.031*"mass" + 0.028*"disk" + 0.018*"formation" + 0.017*"model"
Topic 7: 0.025*"outgoing" + 0.023*"oblique" + 0.017*"penrose" + 0.014*"unfolding" + 0.011*"dt"
Topic 8: 0.023*"particle" + 0.014*"surface" + 0.012*"liquid" + 0.012*"simulation" + 0.012*"fluid"
Topic 9: 0.023*"redshift" + 0.022*"survey" + 0.012*"data" + 0.012*"cosmic" + 0.012*"galaxy"
Topic 10: 0.298*

In [26]:
#Hàm tìm topics của 1 bag of words và lấy ra num chủ để có tỉ lệ cao nhất có xác suất không kém no_lower_than
#bag of words này sẽ là 1 bow của abstract tìm chủ đề và xếp bài đó vào danh sách chủ để
#bag of words này cũng sẽ là đầu vào, thông tin người dùng nhập vào thanh search để tìm kiếm
def bow_to_topicid(bow,no_lower_than = 0.2,num=3):
    p_topics = lda_model.get_document_topics(bow, minimum_probability = no_lower_than)
    top = sorted(p_topics, key=lambda x: x[1], reverse=True)[:num]
    return top

In [25]:
# hàm tìm kiếm và trả về xác suất chủ đề của chuỗi kí tự so với dữ liệu của lda
def search_to_topicid(search,no_lower_than = 0.2,num=3):
    
    preprocess_search = preprocess(search)
    bow_search = dictionary.doc2bow(preprocess_search)
    top = bow_to_topicid(bow_search,no_lower_than,num)
    return top


In [14]:
# bước chuẩn bị để tạo một dictionary convert từ topic ra danh sách các bài báo thuộc chủ đề đó.
id = df['id'].values
id.shape


In [15]:
#dictionary để chuyển từ id của bài báo thành BoW của bài báo đó
id_to_bow = {}
for i in range(len(bow_corpus)):
    id_to_bow[id[i]] = bow_corpus[i]

In [16]:
# save id_to_bow
import pickle 

with open('new_id_to_bow.pkl', 'wb') as f:
    pickle.dump(id_to_bow, f)
        


In [19]:
#load id_to_bow
import pickle 
with open('new_id_to_bow.pkl', 'rb') as f:
    loaded_id_to_bow = pickle.load(f)

id_to_bow = loaded_id_to_bow

In [17]:
#tạo topicid_to_ids, dictionary tra cứu từ topicid thành list các bài báo thuộc chủ đề đó

values = [[] for _ in range(num_topics)]

for id,bow in id_to_bow.items():
    top = bow_to_topicid(bow)
    for topic_id, v in top:
        values[topic_id].append(id)

topicid_to_ids = {}

for i in range(len(values)):
    topicid_to_ids[i] = values[i]

In [18]:
#save topicid_to_ids
import pickle 

with open('new_topicid_to_ids.pkl', 'wb') as f:
    pickle.dump(topicid_to_ids, f)
        

In [4]:
#load topicid_to_ids
import pickle 
with open('new_topicid_to_ids.pkl', 'rb') as f:
    topicid_to_ids = pickle.load(f)

In [3]:
# từ thông tin id của bài báo để lấy về các thông tin đầy đủ của bài báo

def id_to_contents(id):
    result = df.loc[df['id'] == id]
    return result

In [23]:
# hàm trả về các bài báo có chủ đề liên quan đến chủ đề nhận được
def search_results(top,num_result = 10, topics = 3):
    results = df.iloc[:2]
    results = results.drop([0,1])


    
    for topicid, prob in top[:topics]:
        ids = topicid_to_ids[topicid]
       
        for id in ids:
            next = id_to_contents(id)
           
            if num_result == 0 :
                break
            elif next is None:
                break
            else:
                num_result = num_result -1
            results = pd.concat([results, next],axis = 0)
  

    return results
   



In [1]:
# hàm trả về các bài báo có chủ đề liên quan đến chuỗi kí tự nhận được
def search(text:str,no_lower_than=0.2,num=3,num_result=10,topics=3):
    top = search_to_topicid(text,no_lower_than,num)
    results = search_results(top,num_result,topics)
    return results
    

In [27]:
# vì một lý do nào đó, lúc chạy cái lệnh này thì với "Investing" thì vẫn có kết quả là chủ đề liên quan, nhưng với cái bản mới nhất thì lại không có
search("investing")

,id,title,doi,categories,abstract,update_date,authors_parsed
9,2406.04057,Overwhelmed Software Developers,10.1109/MS.2024.3374821,cs.SE cs.CY,We have conducted a qualitative psychology s...,2024-06-07,"[['Michels', 'Lisa-Marie', ''], ['Petkova', 'A..."
10,2406.04066,Requirements for Organizational Resilience: En...,10.1109/MS.2024.3386035,cs.SE cs.CY,Can the right requirements boost developer s...,2024-06-07,"[['Borg', 'Markus', ''], ['Graziotin', 'Daniel..."
27,2406.03859,From operculum and body tail movements to diff...,10.1016/j.compag.2020.105531,cs.CV q-bio.PE,The AEFishBIT tri-axial accelerometer was ex...,2024-06-07,"[['Ferrer', 'Miguel A.', ''], ['Calduch-Giner'..."
44,2209.12222,Efficient Wrong-Way Risk Modelling for Funding...,10.1142/S0219024924500109,q-fin.CP q-fin.MF q-fin.RM,Wrong-Way Risk (WWR) is an important compone...,2024-06-07,"[['van der Zwaard', 'T.', ''], ['Grzelak', 'L...."
50,2406.035,"Impact of aleatoric, stochastic and epistemic ...",10.1016/j.ijpe.2022.108626,stat.AP q-fin.RM,"In construction projects, contingency reserv...",2024-06-07,"[['Curto', 'David', ''], ['Acebes', 'Fernando'..."
52,2406.03547,Learning from the present for the future: the ...,10.1016/j.ascom.2024.100835,astro-ph.IM,The Forschungszentrum Juelich has been hosti...,2024-06-07,"[['Manzano', 'C.', ''], ['Miskolczi', 'A.', ''..."
62,2305.0378,Boldness-Recalibration for Binary Event Predic...,10.1080/00031305.2024.2339266,stat.ME stat.ML,Probability predictions are essential to inf...,2024-06-07,"[['Guthrie', 'Adeline P.', ''], ['Franck', 'Ch..."
71,2401.04206,Effects of Multimodal Explanations for Autonom...,10.1038/s41598-024-62052-9,cs.HC cs.AI cs.RO,Advances in autonomous driving provide an op...,2024-06-07,"[['Kaufman', 'Robert', ''], ['Costa', 'Jean', ..."
75,2305.10091,"Multi-Agent Reinforcement Learning: Methods, A...",10.1109/TIV.2024.3408257,cs.AI,Multi-agent reinforcement learning (MARL) is...,2024-06-07,"[['Zhou', 'Ziyuan', ''], ['Liu', 'Guanjun', ''..."
77,2311.13063,From Classification to Clinical Insights: Towa...,10.1145/3659604,cs.AI,Passively collected behavioral health data f...,2024-06-07,"[['Englhardt', 'Zachary', ''], ['Ma', 'Chengqi..."


In [90]:
id_to_contents('2406.04057')

,id,title,doi,categories,abstract,update_date,authors_parsed
9,2406.04057,Overwhelmed Software Developers,10.1109/MS.2024.3374821,cs.SE cs.CY,We have conducted a qualitative psychology s...,2024-06-07,"[['Michels', 'Lisa-Marie', ''], ['Petkova', 'A..."
